### Function
- Search the number of airline between to distinct cities
- Show the airline(s)

In [75]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.faker import Faker
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ChartType, SymbolType
import numpy as np
import pandas as pd

In [76]:
Airplane_city = pd.read_csv('Airplane_city.csv')
Airplane_route = pd.read_csv('Airplane_route.csv')
Airplane_airport = pd.read_csv('Airplane_airport.csv')

In [77]:
Airplane_city['city'].unique()

array(['阿克苏', '阿勒泰', '安庆', '包头', '保山', '北海', '北京', '百色', '长春', '长沙', '长治',
       '常德', '常州', '成都', '赤峰', '重庆', '昌都', '朝阳', '达州', '大理', '大连', '丹东',
       '东营', '敦煌', '大同', '大庆', '鄂尔多斯', '恩施', '福州', '阜阳', '赣州', '格尔木',
       '广州', '贵阳', '桂林', '哈尔滨', '海口', '哈密', '海拉尔', '邯郸', '汉中', '杭州', '合肥',
       '和田', '黑河', '呼和浩特', '黄山', '济南', '济宁', '佳木斯', '嘉峪关', '锦州', '井冈山',
       '景德镇', '鸡西', '喀什', '库尔勒', '昆明', '克拉玛依', '拉萨', '兰州', '丽江', '连云港',
       '临沧', '临沂', '柳州', '泸州', '洛阳', '满洲里', '芒市', '绵阳', '牡丹江', '梅县', '漠河',
       '南昌', '南充', '南京', '南宁', '南通', '南阳', '宁波', '攀枝花', '齐齐哈尔', '青岛',
       '衢州', '三亚', '上海', '深圳', '沈阳', '石家庄', '塔城', '太原', '台州', '天津', '通辽',
       '铜仁', '天水', '万州', '威海', '潍坊', '温州', '乌海', '乌兰浩特', '乌鲁木齐', '无锡',
       '武汉', '武夷山', '梧州', '西安', '西昌', '西宁', '西双版纳', '锡林浩特', '厦门', '香格里拉',
       '徐州', '兴义', '烟台', '延安', '延吉', '盐城', '伊宁', '宜宾', '宜昌', '义乌', '银川',
       '永州', '榆林', '运城', '伊春', '玉树', '湛江', '张家界', '昭通', '郑州', '舟山', '珠海',
       '芷江'], dtype=object)

In [78]:
name = Airplane_airport[Airplane_airport['city'] == '北京']['airport'].values
print(name)

['首都机场' '大兴机场']


In [79]:
'/'.join(name)

'首都机场/大兴机场'

In [80]:
# Airplane_route[
#     Airplane_route["departure_city"] == "北京"
# ].to_records(index=False).tolist()

In [81]:
def Airline_search(Airport_list, Route_list):
    MAP = (
        Geo(init_opts=opts.InitOpts(
            width="1000px", 
            height="730px"))
        .add_coordinate("芒市", 98.38210533, 24.37681138)
        .add_coordinate("加格达奇", 124.1316504, 50.4057919)
        .add_schema(maptype="china",
                    itemstyle_opts=opts.ItemStyleOpts(color="#323c48", border_color="#111"))
        .add(
            "",
            Airport_list,
            type_=ChartType.EFFECT_SCATTER,
            color="green",
        )
        .add(
            "Airline",
            Route_list,
            type_=ChartType.LINES,
            effect_opts=opts.EffectOpts(
                symbol=SymbolType.ARROW, symbol_size=6, color="blue"
            ),
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="全国主要城市航班路线和机场"))
    )
    return MAP

In [82]:
begin_city = "深圳"
end_city = 0

def Find_airport(city):
    airport_name = Airplane_airport[Airplane_airport['city'] == city]['airport'].values
    return '/'.join(airport_name)

def Create_Airport_list(city, route_list, type):
    # 1: begin city, 0: end city
    Num_list = [(city, Find_airport(city))]
    for route in route_list:
        route_turple = (route[type], Find_airport(route[type]))
        Num_list.append(route_turple)
    return Num_list

if begin_city and end_city:
    Airline_Num = len(
        Airplane_route[
            (Airplane_route["departure_city"] == begin_city)
            & (Airplane_route["landing_city"] == end_city)
        ]
    )
    begin_airport = Find_airport(begin_city)
    end_airport = Find_airport(end_city)
    if Airline_Num:
        c = Airline_search([(begin_city, begin_airport), 
                            (end_city, end_airport)], 
                            [(begin_city, end_city)])
    else:
        print("No such route from {} to {}".format(begin_city, end_city))
else:
    if begin_city == 0 and end_city != 0:
        Route_list = (
            Airplane_route[Airplane_route["landing_city"] == end_city]
            .to_records(index=False)
            .tolist()
        )
        Airport_list = Create_Airport_list(end_city, Route_list, 0)
        c = Airline_search(Airport_list, Route_list)
    elif end_city == 0 and begin_city != 0:
        Route_list = (
            Airplane_route[Airplane_route["departure_city"] == begin_city]
            .to_records(index=False)
            .tolist()
        )
        Airport_list = Create_Airport_list(begin_city, Route_list, 1)
        c = Airline_search(Airport_list, Route_list)
    elif begin_city == 0 and end_city == 0: # All routes
        print("Input error!")
c.render_notebook()